In [1]:
import os
import cv2
import PIL
import numpy as np
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm 
from tensorflow import keras
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale = 1.4)
from sklearn.utils import shuffle
from keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
class_names = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
class_name_labels = {class_name : i for i, class_name in enumerate(class_names)}

number_of_classes = len(class_names)
print(class_name_labels)
IMAGE_SIZE = (180, 180)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [3]:
def load_data():
    DIRECTORY = r"D:\College\Projects\Flower_Recognition"
    CATEGORY = ['training_set', 'test_set']
    
    output = []
    
    for category in CATEGORY:
        path = os.path.join(DIRECTORY, category)
        images = []
        labels = []
        
        print('loading {}'.format(category))
        
        for folder in os.listdir(path):
            label = class_name_labels[folder]
            
            for file in os.listdir(os.path.join(path, folder)):
                img_path = os.path.join(os.path.join(path, folder), file)
                
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)
                
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')
            
        output.append((images, labels))
        
    return output


In [4]:
(training_images, training_labels), (test_images, test_labels) = load_data()

loading training_set
loading test_set


In [5]:
training_images, training_labels = shuffle(training_images, training_labels, random_state = 25)

In [11]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(180,
                                  180,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [12]:
num_classes = len(class_names)

model = Sequential([
     
  data_augmentation,

  layers.Rescaling(1./255),
  
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  

  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

In [24]:

history = model.fit(training_images, training_labels, batch_size = 128, epochs = 22, validation_split= 0.2)

Epoch 1/22
23/23 [==============================] - 36s 1s/step - loss: 0.4867 - accuracy: 0.8119 - val_loss: 0.9054 - val_accuracy: 0.6791
Epoch 2/22
23/23 [==============================] - 33s 1s/step - loss: 0.4440 - accuracy: 0.8377 - val_loss: 0.8653 - val_accuracy: 0.6928
Epoch 3/22
23/23 [==============================] - 31s 1s/step - loss: 0.4323 - accuracy: 0.8308 - val_loss: 0.8565 - val_accuracy: 0.6804
Epoch 4/22
23/23 [==============================] - 31s 1s/step - loss: 0.4173 - accuracy: 0.8425 - val_loss: 0.9282 - val_accuracy: 0.6529
Epoch 5/22
23/23 [==============================] - 32s 1s/step - loss: 0.4182 - accuracy: 0.8387 - val_loss: 0.7854 - val_accuracy: 0.7176
Epoch 6/22
23/23 [==============================] - 33s 1s/step - loss: 0.3699 - accuracy: 0.8615 - val_loss: 0.8977 - val_accuracy: 0.7066
Epoch 7/22
23/23 [==============================] - 32s 1s/step - loss: 0.4048 - accuracy: 0.8522 - val_loss: 0.8377 - val_accuracy: 0.7135
Epoch 8/22
23/23 [==

In [ ]:
# model.save('final_model2.h5')

## Evaluating Model

In [23]:
result = model.evaluate(test_images, test_labels, batch_size=128)

6/6 [==============================] - 1s 162ms/step - loss: 0.9271 - accuracy: 0.6604
